In [79]:
import csv
import subprocess
import math
import json
import os
import shlex
import cv2 
import shutil
from tqdm import tqdm
import numpy as np

In [46]:
def cleardir(dirpath):
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    try:
        os.mkdir(dirpath)
    except OSError:
        print ("Creation of the directory %s failed" % dirpath)
    else:
        print ("Successfully created the directory %s " % dirpath)

In [70]:
# https://github.com/c0decracker/video-splitter/blob/master/ffmpeg-split.py

def get_video_length(filename):

    output = subprocess.check_output(("ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename)).strip()
    video_length = int(float(output))
    print ("Video length in seconds: "+str(video_length))

    return video_length

def ceildiv(a, b):
    return int(math.ceil(a / float(b)))

def split_by_seconds(filename, split_length, vcodec="copy", acodec="copy",
                     extra="", video_length=None, **kwargs):
    if split_length and split_length <= 0:
        print("Split length can't be 0")
        raise SystemExit

    if not video_length:
        video_length = get_video_length(filename)
    split_count = ceildiv(video_length, split_length)
    if(split_count == 1):
        print("Video length is less then the target split length.")
        raise SystemExit

#     split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec, "-acodec", acodec] + shlex.split(extra)
    split_cmd = ["ffmpeg"] + shlex.split(extra)
    try:
        filebase = ".".join(filename.split(".")[:-1])
        shortname = filebase.split('/')[-1]
        fileext = filename.split(".")[-1]
    except IndexError as e:
        raise IndexError("No . in filename. Error: " + str(e))
    
    path = "vid_short/" + shortname
    cleardir(path)
        
    for n in range(0, split_count):
        split_args = []
        if n == 0:
            split_start = 0
        else:
            split_start = split_length * n

#         split_args += ["-ss", str(split_start), "-t", str(split_length),
#                        filebase + "-" + str(n+1) + "-of-" + \
#                         str(split_count) + "." + fileext]
        split_args += ["-y", "-ss", str(split_start), "-t", str(split_length), "-i", filename, 
                       "-vcodec", vcodec, "-acodec", acodec,
                       "vid_short/" + shortname + "/" + shortname + str(n+1) + "." + fileext]
        print ("About to run: "+" ".join(split_cmd+split_args))
        subprocess.check_output(split_cmd+split_args)

In [61]:
# https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/
# Program To Read video 
# and Extract Frames 
  
# Function to extract frames 
def FrameCapture(inpath, outfolder, div = 24): 
      
    cleardir(outfolder)
    
    # Path to video file 
    vidObj = cv2.VideoCapture(inpath) 
  
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
  
    while success: 
        vidObj.set(cv2.CAP_PROP_POS_MSEC,(count*1000/div))
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
        if not success:
            break
        # Saves the frames with frame-count 
        cv2.imwrite((outfolder + "/" + "frame%d.jpg") % count, image) 
  
        count += 1
  


In [62]:
def rejoin(folder, pattern, outfile, fps):
    cmd = "ffmpeg -r " + str(fps) + " -i " + folder + '/' + pattern + " -vcodec mpeg4 -y " + outfile
    print(cmd)
    os.system(cmd)
#     os.system("ffmpeg -r 1 -i frame%01d.jpg -vcodec mpeg4 -y movie.mp4")

In [76]:
def edge_promoting(root, save):
    file_list = os.listdir(root)
    if not os.path.isdir(save):
        os.makedirs(save)
    kernel_size = 5
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    gauss = cv2.getGaussianKernel(kernel_size, 0)
    gauss = gauss * gauss.transpose(1, 0)
    n = 1
    for f in tqdm(file_list):
        rgb_img = cv2.imread(os.path.join(root, f))
        gray_img = cv2.imread(os.path.join(root, f), 0)
        rgb_img = cv2.resize(rgb_img, (256, 256))
        pad_img = np.pad(rgb_img, ((2,2), (2,2), (0,0)), mode='reflect')
        gray_img = cv2.resize(gray_img, (256, 256))
        edges = cv2.Canny(gray_img, 100, 200)
        dilation = cv2.dilate(edges, kernel)

        gauss_img = np.copy(rgb_img)
        idx = np.where(dilation != 0)
        for i in range(np.sum(dilation != 0)):
            gauss_img[idx[0][i], idx[1][i], 0] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 0], gauss))
            gauss_img[idx[0][i], idx[1][i], 1] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 1], gauss))
            gauss_img[idx[0][i], idx[1][i], 2] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 2], gauss))

        result = np.concatenate((rgb_img, gauss_img), 1)

        cv2.imwrite(os.path.join(save, str(n) + '.png'), result)
        n += 1

In [71]:
filename = "Jared.mp4"
split_by_seconds("vid_full/" + filename, 10)

Video length in seconds: 70
Successfully created the directory vid_short/Jared 
About to run: ffmpeg -ss 0 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared1.mp4
About to run: ffmpeg -ss 10 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared2.mp4
About to run: ffmpeg -ss 20 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared3.mp4
About to run: ffmpeg -ss 30 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared4.mp4
About to run: ffmpeg -ss 40 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared5.mp4
About to run: ffmpeg -ss 50 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared6.mp4
About to run: ffmpeg -ss 60 -t 10 -i vid_full/Jared.mp4 -vcodec copy -acodec copy vid_short/Jared/Jared7.mp4


In [77]:
FrameCapture("vid_short/Jared/Jared2.mp4", "vid_splice/Jared2")

Successfully created the directory vid_splice/Jared2/ 


KeyboardInterrupt: 

In [73]:
rejoin("vid_splice/Jared2", "frame%01d.jpg", "vid_join/Jared2.mp4" ,24)

ffmpeg -r 24 -i vid_splice/Jared2/frame%01d.jpg -vcodec mpeg4 -y vid_join/Jared2.mp4


In [80]:
edge_promoting('vid_splice/Jared2', 'vid_splice/Jared2Fuzzy')

100%|██████████| 22/22 [00:18<00:00,  1.22it/s]


In [81]:
FrameCapture("vid_data/join/Jared2.mp4", "")

Creation of the directory  failed
